In [1]:
import numpy as np
import torch as t
import torch_geometric as pyg
import networkx as nx
from matplotlib import pyplot as plt

In [44]:
cora = pyg.datasets.planetoid.Planetoid('./data', name="cora")

In [45]:
print(cora.edge_index)

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])


In [46]:
n_edges = np.array(range(cora.edge_index.shape[1]))
n_edges = np.random.permutation(n_edges)

limit = int(len(n_edges)*0.8)
train = n_edges[:limit]
test = n_edges[limit:]

train, test

(array([  355,  2999,  7819, ..., 10448,  6552,  9315]),
 array([5776, 4245, 9093, ...,  993, 7561, 4906]))

In [47]:
Adj = pyg.utils.to_dense_adj(cora.edge_index)[0]
print(Adj -np.eye(Adj.shape[0]))
AdjProcessed = 1 - Adj - np.eye(Adj.shape[0])
neg_u, neg_v = np.where(AdjProcessed != 0)

tensor([[-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0., -1.,  1.,  ...,  0.,  0.,  0.],
        [ 0.,  1., -1.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ..., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0., -1.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  1., -1.]], dtype=torch.float64)


C:\Users\XPD\AppData\Local\Temp\ipykernel_15468\3158052384.py:2: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(Adj -np.eye(Adj.shape[0]))
C:\Users\XPD\AppData\Local\Temp\ipykernel_15468\3158052384.py:3: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  AdjProcessed = 1 - Adj - np.eye(Adj.shape[0])


In [50]:
neg_ids = np.random.choice(len(neg_u), len(n_edges))
train_neg_u = neg_u[neg_ids[:limit]]
train_neg_v = neg_v[neg_ids[:limit]]
print(train_neg_u[:10], train_neg_v[:10])
test_neg_u = neg_u[neg_ids[limit:]]
test_neg_v = neg_v[neg_ids[limit:]]

print(test_neg_u[:10], test_neg_v[:10])

[1852  803 1260  818 2636  444 1729 1153 1477  524] [2143  967  427   39  621 2107  991  349 2242 1988]
[1806 1131  444 1073  287 2690 1320 2567 2442 2147] [1926   72 1196  610 2457 2003 2214  889 1501  807]


In [70]:
train_cora = t.stack([
    cora.edge_index[0][train],
    cora.edge_index[1][train]
])
train_cora

tensor([[1658, 2093, 2388,  ..., 2666, 2034, 1138],
        [  88,  741, 1914,  ..., 2665, 1654, 2250]])

In [75]:
class Net(t.nn.Module):
    def __init__(self, insize, outsize):
        super(Net).__init__()
        self.gsage1 = pyg.nn.SAGEConv(in_channels=insize, out_channels=outsize)        
        self.gsage2 = pyg.nn.SAGEConv(in_channels=outsize, out_channels=outsize)
        self.hsize = hsize
        self.outsize = outsize

    def forward(self, data):
        x = data.x
        edges = data.edge_index
        h = self.gsage1(x, edges)
        h = t.nn.functional.leaky_relu(h)
        h = self.gsage2(h, edges)
        return h        

In [97]:
train_pos = t.stack([cora.edge_index[0][train], cora.edge_index[1][train]])
train_neg = t.stack([t.tensor(train_neg_u), t.tensor(train_neg_v)])

test_pos = t.stack([cora.edge_index[0][test], cora.edge_index[1][test]])
test_neg = t.stack([t.tensor(test_neg_u), t.tensor(test_neg_v)])

print(train_pos[:10], train_neg[:10], test_pos[:10], test_pos[:10])

tensor([[1658, 2093, 2388,  ..., 2666, 2034, 1138],
        [  88,  741, 1914,  ..., 2665, 1654, 2250]]) tensor([[1852,  803, 1260,  ..., 1311, 2321, 1878],
        [2143,  967,  427,  ..., 1945, 1282, 1319]]) tensor([[1537,  982,  964,  ..., 1274, 1701,  505],
        [1455, 1091, 2198,  ...,  239, 1859, 1258]]) tensor([[1537,  982,  964,  ..., 1274, 1701,  505],
        [1455, 1091, 2198,  ...,  239, 1859, 1258]])
